# Objetos y Modulos

## Definición a la API pública

https://platzi.com/clases/1378-python-practico/14182-definicion-a-la-api-publica/

Dejaremos nuestro archivo *main.py* quieto. Solo añadiremos una linea, para indicarle se ejecute python de manera implicita:

        #!/usr/bin/env python

Hasta ahora hemos trabajado en un solo archivo. Sin embargo en la industria, es comun usar muchos archivos de tal forma que el software sea legible, donde cada uno de los archivos tendra una funcione especifica

Crearemos un nuevo directorio: **app-platzi-ventas**. Y crearemos una estructura de archivos:

- pv.py: es el punto de entrada

![](https://i.imgur.com/2mBSbUL.png)

En Python, un subdirectorio, verbigracia *clients* se conoce como **modulos**.

### ¿Porque Instalar una aplicacion?

Generalmente cuando usas cualquier aplicacion de linea de comandos, no tienes que colocar:

    python3 git logs ❌

Simplemente ejecutamos el comando y ya. Lo que queremos es poder ejecutar nuestra aplicacion simplemente asi:

    pv

Nuestra aplicacion se llama **pv**(platzi ventas).

#### Empezaremos por el archivo pv.py 

Aqui solo ira el punto de entrada


![](https://i.imgur.com/dcMgdRT.png)


- Para registrar los comandos, donde *cli* hace referencia a la funcion,(da un poco de legibilidad al codigo, de otra forma escribiriamos *client_commands.clients*) :

    cli.add_command(client_commands.all)

- Se importa del modulo clients *commands* que se define a continuacion

### Definiendo comandos basicos

Por ahora, solo esta el comando *search*. 

![](https://i.imgur.com/P980C2p.png)

- all es un alias de clientes -- all apunta a la funcion clients


### Implementar setup.py

Para invocar directamente la *App* de linea de comandos, sin tener que llamar al interprete de python.

- setuptools es una libreria que tiene muchas funcionalidades, pero solo usaremos el modulo *setup* que parece bastante complejo

*"Setuptools is a collection of enhancements to the Python distutils that allow developers to more easily build and distribute Python packages, especially ones that have dependencies on other packages."*


![](https://i.imgur.com/VFT4iEY.png)

#### Instalacion de la App

- EN un ambiente virtual
- Instalarla, no olvidar el puntico. La opcion **--editable** significa los cambios que se hagan se veran reflejados, asi este instalada. 

        pip install --editable .

![](https://i.imgur.com/R9I7RBV.png)

verificar con:

    which pv

Salida:

    https://i.imgur.com/R9I7RBV.png    

#### Verificando las funciones de ayuda

![](https://i.imgur.com/HbSK7jn.png)

        

## Pasando el contexto

Desafortunadamente la explicacion fue muy deficiente, y la documentacion es algon enredada. Asi que hare mi mejor esfuerzo:

En **pv.py**: 

- Declararemos una constante con el nombre del archivo:

        CLIENTS_TABLE = '.clients.csv'

- Se hace uso del decorador @click.pass_context, que al usarlo devuelve un objeto contexto(ctx), Que inicializamos como un diccionario vacio.

- Asignaremos un campo llave:valor al contexto con el nombre del archivo 

        ctx.obj['clients_table'] = CLIENTS_TABLE

![](https://i.imgur.com/3rPHbUs.png)


En **commands.py**

![](https://i.imgur.com/ELuTorJ.png)

- con el decorador @click.pass_context le pasamos el contexto.
- asignamos esta variable y miramos que hay dentro de ella:

        client_service = ctx.obj['clients_table']

- ejecutamos la aplicacion:

        pv clients list 
![](https://i.imgur.com/5BrC7DX.png)

No es nada raro. Solo es una referencia al archivo desde donde se van a leer los datos. 😊

### Primera implementacion del comando list

Todavia no se hara uso de POO. Se hara desde *commands.py*
- Agregando... 

![](https://i.imgur.com/HHViXaG.png)

- Solo cambiaremos por ahora el nombre de client_service a client_table
- Se agrega la funcion _load_data_from_csv como en main.py
- Se hace uso de la libreria *tabulate* para mostrarlo en forma de tabla 

![](https://i.imgur.com/h6RuPxx.png)

Asi ejecutamos:

    pv clients list

![](https://i.imgur.com/W2cS3y4.png)

Como ejercicio adicional, agrega un campo para el salario 

## Clients y segunsa implementacion del comando list

Hasta ahora solo tenemos tres archivos que componen la aplicacion:

- setup.py
- commands.py
- pv.py

- Crearemos dos nuevos archivos:

        touch clients/models.py 
    
        touch clients/services.py 

¿Porque esta estructura?

Normalmente en un App del mundo real, se tiene que dividir entre la interfax, es decir como interactura el software con el exterior, en el caso de nuestra aplicacion son los *comandos*, si estuvieramos en una aplicacion WEB seria la interfax grafica, si estuvieramos en una API de servidor serian los *endpoints*

Luego tendriamos la logica del negocio, esa la manejaremos en servicios. Y por ultimo tenemos las abstracciones, y objetos sobre los cuales estamos interactuando, en este caso serian los clientes y los manejamos en modelos.

#### Declarando una clase

Dentro de **models.py** declararamos una clase, que por ahora no la vamos a usar, ni tampoco la funcion *to_dict*. Solo usaremos el metodo estatico por ahora. 

![](https://i.imgur.com/DhtOpxZ.png)

Se puede declarar un metodo estatico con el decorador **@staticmethod**. Un metodo estatico es un metodo que se puede ejecutar sin necesidad de una instancia de clase. Aqui solo guardaremos el schema:

Antes:

        CLIENT_SCHEMA = ['name', 'company', 'position', 'salary']


Despues: 

    @staticmethod
    def schema():
        return ['name', 'company', 'position', 'salary']

Observa los metodos estaticos no requieren *el keyword self*


#### Modificando el Servicio

Hasta ahora, dentro del mismo archivo *commands.py* manejamos los *endpoints* por decirlo de alguna manera de nuestra Aplicacion, y tambien la busqueda, o lo que sirve la aplicacion.

Independizaremos los servicios de los endpoints.

Iremos a **services.py** donde guardaremos nuestra logica de negocio: que significa ser update, o que significa delete, o list, o create. Y lo declararemos como una clase. 

Por ahora solo implementaremso el metodo para listar todos los elementos.

![](https://i.imgur.com/qggmqEM.png)

- **reader** es un iterable, pero lo podemos convertir en una lista
- No usaremos mas la funcion *_load_data_from_csv*
- **render-list** es un metodo estatico para renderizar la tabla de forma bonita




#### Modificando commands.py

    from clients.services import ClientService

Y la funcion

    @clients.command()
    @click.pass_context
    def list(ctx):
        '''List all the clients'''
        client_table = ctx.obj['clients_table']
        client_service = ClientService(client_table)
        clients_list = client_service.get_clients_list()
        ClientService.render_list(clients_list)

Guardamos nuestro trabajo en git. Como "segunda implementacion funcion list"

## Servicios: Logica de Negocio de la Aplicacion

Ya hemos implementado el comando *list*. ¿Que tal si seguimos implementando las demas funciones?

### Implementando la funcion de busqueda

![](https://i.imgur.com/OWdC1Oi.png)

- Se crear el comando *search* que tiene un argumento: username, y una opcion(byname) si se va a realizar la busqueda por nombre. Notar que es un booleano y que el valor por defecto es falso. Bueno deberia ser al reves, pero funciona.

- se implementa el funcion *search_client_by_name*

![](https://i.imgur.com/G4FsCFv.png)

Como ves retorna el diccionario correspondiente a donde esta almacenada la busqueda, de otra forma retorna None.

Ahora ejecutemos el comando:

    pv clients search joe

![](https://i.imgur.com/9EZAz0H.png)

Si queremos ahora ir a la parte que no esta implementada, para cambiar el valor de la opcion *byname* a verdadero, simplemente

    pv clients search joe --byname

Y aparecera un mensaje que *no esta implementado*


#### 